In [ ]:
import os
import re
import pandas as pd
import difflib
import nltk
from collections import Counter

files = [
    "textFooler/textfooler_results_all.csv",
    "PWWS/pwws_results_all.csv",
    "textBugger/textbugger_results_all.csv",
]

dfs = []
for path in files:
    if not os.path.exists(path):
        raise FileNotFoundError(f"Nu am găsit fișierul: {path}")
    df = pd.read_csv(path)
    dfs.append(df)
results = pd.concat(dfs, ignore_index=True)

results = results[results["result_type"] == "Successful"].copy()

def clean_markers(text):
    return re.sub(r"\[\[(.*?)\]\]", r"\1", text)

results["orig_clean"] = results["original_text"].apply(clean_markers)
results["pert_clean"] = results["perturbed_text"].apply(clean_markers)

def diff_pairs(orig, pert):
    o, p = orig.split(), pert.split()
    diff = list(difflib.ndiff(o, p))
    pairs = []
    i = 0
    while i < len(diff):
        if diff[i].startswith("- "):
            o_word = diff[i][2:]
            j = i + 1
            while j < len(diff) and not diff[j].startswith("+ "):
                j += 1
            if j < len(diff):
                p_word = diff[j][2:]
                pairs.append((o_word, p_word))
                i = j
        i += 1
    return pairs

results["diff_pairs"] = results.apply(
    lambda row: diff_pairs(row["orig_clean"], row["pert_clean"]), axis=1
)
results["num_changes"] = results["diff_pairs"].apply(len)

avg_changes = results["num_changes"].mean()

all_pairs = [pr for sub in results["diff_pairs"] for pr in sub]
top10 = Counter(all_pairs).most_common(10)

orig_words = [o for o,_ in all_pairs]
pos_tags = nltk.pos_tag(orig_words)
pos_counts = Counter(tag for _, tag in pos_tags).most_common()

print(f"\n→ Average number of word changes per example: {avg_changes:.2f}\n")

print("→ Top 10 replacement pairs (orig → new):")
print("Count | Original → New")
print("-"*30)
for (o,p),cnt in top10:
    print(f"{cnt:5d} | {o} → {p}")
print()

print("→ POS distribution of replaced (original) words:")
print("Count | POS tag")
print("-"*25)
for tag, cnt in pos_counts:
    print(f"{cnt:5d} | {tag}")



→ Average number of word changes per example: 33.91

→ Top 10 replacement pairs (orig → new):
Count | Original → New
------------------------------
  293 | said → aforesaid
  261 | said → stated
  239 | The → Both
  177 | I → me
  173 | know → recognise
  162 | new → novel
  162 | The → Per
  154 | said. → aforesaid.
  149 | The → he
  148 | said → enunciate

→ POS distribution of replaced (original) words:
Count | POS tag
-------------------------
57589 | NN
42459 | NNP
34030 | JJ
17049 | NNS
14758 | VBD
10966 | VBG
10542 | RB
 9604 | VBP
 6153 | VBN
 5391 | VB
 4606 | VBZ
 4222 | IN
 4156 | CD
 2025 | PRP
 1855 | DT
 1413 | MD
  779 | JJS
  569 | JJR
  490 | NNPS
  396 | POS
  327 | RBR
  316 | CC
  231 | PRP$
  154 | FW
  148 | RP
  123 | TO
  122 | WRB
  118 | WP
   50 | EX
   35 | WDT
   34 | WP$
   22 | RBS
   17 | ''
   13 | PDT
   11 | $


In [ ]:
import os
import re
import pandas as pd
import difflib
import nltk
from collections import Counter, defaultdict

# nltk.download("averaged_perceptron_tagger")  

files = [
    "textFooler/textfooler_results_all.csv",
    "PWWS/pwws_results_all.csv",
    "textBugger/textbugger_results_all.csv",
]

def clean_markers(text):
    return re.sub(r"\[\[(.*?)\]\]", r"\1", text)

def extract_replacements(orig, pert):
    o_tokens = orig.split()
    p_tokens = pert.split()
    diff = list(difflib.ndiff(o_tokens, p_tokens))
    pairs = []
    i = 0
    while i < len(diff):
        if diff[i].startswith("- "):
            original = diff[i][2:]
            j = i + 1
            while j < len(diff) and not diff[j].startswith("+ "):
                j += 1
            if j < len(diff):
                replaced = diff[j][2:]
                pairs.append((original, replaced))
                i = j
        i += 1
    return pairs

replacement_counter = defaultdict(Counter)

for path in files:
    if not os.path.exists(path):
        print(f"Nu găsesc fișierul: {path}")
        continue

    df = pd.read_csv(path)
    df = df[df["result_type"] == "Successful"]

    for _, row in df.iterrows():
        orig = clean_markers(row["original_text"])
        pert = clean_markers(row["perturbed_text"])

        pairs = extract_replacements(orig, pert)
        if not pairs:
            continue

        orig_words = [orig for orig, _ in pairs]
        pos_tags = nltk.pos_tag(orig_words)

        for (o_word, p_word), (_, tag) in zip(pairs, pos_tags):
            if tag.startswith("VB"):
                replacement_counter[o_word][p_word] += 1

print("\nTop 10 substantive înlocuite și înlocuirile lor frecvente:\n")
for word, repls in Counter({k: sum(v.values()) for k, v in replacement_counter.items()}).most_common(20):
    print(f"{word} ->")
    for rep, cnt in replacement_counter[word].most_common(3):
        print(f"    {cnt}x {rep}")



Top 10 substantive înlocuite și înlocuirile lor frecvente:

said ->
    293x aforesaid
    261x stated
    148x enunciate
be ->
    101x constituted
    68x constitute
    58x arrive
have ->
    56x possesses
    52x get
    46x ai
has ->
    102x gets
    91x possesses
    85x had
do ->
    62x could
    50x ai
    44x makes
made ->
    56x realised
    33x realise
    20x effected
think ->
    79x reckon
    49x visualise
    22x thin𝒌
make ->
    76x realise
    40x fulfil
    19x accomplish
going ->
    19x gonna
    15x leaving
    14x plump
take ->
    22x pick
    19x involve
    17x wear
told ->
    25x narrate
    17x informs
    17x say
had ->
    48x possesses
    21x gets
    17x ai
get ->
    24x gets
    17x obtain
    15x fetch
see ->
    92x visualise
    32x realise
    24x reckon
know ->
    118x recognise
    63x realise
    6x recognising
According ->
    38x harmonise
    22x concord
    19x Depending
say ->
    13x suppose
    12x telling
    11x sy
seen ->
    3

In [7]:
import pandas as pd
import re

csv_files = [
    "textFooler/textfooler_results_all.csv",
    "textFooler/val_textfooler_results_all.csv",
    "textFooler/test_textfooler_results_all.csv",
    "PWWS/pwws_results_all.csv",
    "PWWS/val_pwws_results_all.csv",
    "PWWS/test_pwws_results_all.csv",
    "textBugger/textbugger_results_all.csv",
    "textBugger/val_textbugger_results_all.csv",
    "textBugger/test_textbugger_results_all.csv"
]

target_words = {"republicans", "Republicans"}
replacements = set()

def extract_marked(text):
    return re.findall(r"\[\[(.*?)\]\]", text)

for file_path in csv_files:
    print(f"Processing: {file_path}")
    try:
        df = pd.read_csv(file_path)
    except Exception as e:
        print(f"Failed to read {file_path}: {e}")
        continue

    for _, row in df.iterrows():
        orig = row.get("original_text", "")
        pert = row.get("perturbed_text", "")
        orig_words = extract_marked(orig)
        pert_words = extract_marked(pert)

        for o, p in zip(orig_words, pert_words):
            if o in target_words:
                replacements.add(p)

print("\nUnique replacements for 'Democrats/democrats':")
for word in sorted(replacements):
    print("-", word)


Processing: textFooler/textfooler_results_all.csv
Processing: textFooler/val_textfooler_results_all.csv
Processing: textFooler/test_textfooler_results_all.csv
Processing: PWWS/pwws_results_all.csv
Processing: PWWS/val_pwws_results_all.csv
Processing: PWWS/test_pwws_results_all.csv
Processing: textBugger/textbugger_results_all.csv
Processing: textBugger/val_textbugger_results_all.csv
Processing: textBugger/test_textbugger_results_all.csv

Unique replacements for 'Democrats/democrats':
- 208
- 2102
- 2O15
- Ameicans
- Assembler
- Barack
- Chief
- Clinton
- Como
- Congressman
- Congresswoman
- Conservatism
- Democrat
- Democrats
- Gingrich
- However
- Msnbc
- Nuevo
- Obama
- Palin
- Politico
- Presidential
- Re
- Reelected
- Repbulicans
- Republican
- Republicanѕ
- Romney
- Senate
- Senators
- Tories
- Tհis
- aiding
- appointing
- ballot
- bulk
- businesses
- changed
- conservative
- decide
- enquired
- integrating
- inѕists
- legislation
- ought
- readies
- reference
- refsal
- republica